# Financial Data and Preprocessing 

## Getting data from Yahoo Finance

In [ ]:
import pandas as pd 
import fix_yahoo_finance as yf

# download data
df_yahoo = yf.download('AAPL', 
                       start='2000-01-01', 
                       end='2010-12-31',
                       progress=False)

print(f'Downloaded {df_yahoo.shape[0]} rows of data.')

df_yahoo.head()

## Getting data from Quandl 

In [ ]:
import pandas as pd 
import quandl

# authentication 
quandl_key = '{key}' # replace {key} with your own API key  
quandl.ApiConfig.api_key = quandl_key

# download data 
df_quandl = quandl.get(dataset='WIKI/AAPL',
                       start_date='2000-01-01', 
                       end_date='2010-12-31')

print(f'Downloaded {df_quandl.shape[0]} rows of data.')

df_quandl.head()

## Getting data from IEX

In [ ]:
import pandas_datareader.data as pdr

# download data 
df_iex = pdr.DataReader(name='AAPL', 
                        data_source='iex', 
                        start='2014-01-01', 
                        end='2018-12-31')

print(f'Downloaded {df_iex.shape[0]} rows of data.')

df_iex.head()

## Getting data from Intrinio

In [ ]:
import intrinio_sdk
import pandas as pd

# authentication and selecting API
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = '{key}'  # replace {key} with your own API key  
security_api = intrinio_sdk.SecurityApi()

# request data
response = security_api.get_security_stock_prices(identifier='AAPL', 
                                                  start_date='2000-01-01',
                                                  end_date='2010-12-31', 
                                                  frequency='daily',
                                                  page_size=10000)

# convert results into pandas DataFrame
df_intrinio = pd.DataFrame([x.to_dict() for x in response.stock_prices]).sort_values('date')
df_intrinio.set_index('date', inplace=True)

print(f'Downloaded {df_intrinio.shape[0]} rows of data.')

df_intrinio.head()

## Converting prices to returns

In [ ]:
import pandas as pd 
import numpy as np
import fix_yahoo_finance as yf

# download data 
df = yf.download('AAPL', 
                 start='2000-01-01', 
                 end='2010-12-31',
                 progress=False)

# remove redundant data and rename column
df = df.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)

# calculate simple and log returns
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

# inspect the return series
df.head()

In [ ]:
import pandas as pd
import quandl

quandl_key = '{key}' # replace {key} with your own API key  
quandl.ApiConfig.api_key = quandl_key

df_all_dates = pd.DataFrame(index=pd.date_range(start='1999-12-31', end='2010-12-31'))
df = df_all_dates.join(df[['adj_close']], how='left') \
                 .fillna(method='ffill') \
                 .asfreq('M')

# download inflation data from Quandl
df_cpi = quandl.get(dataset='RATEINF/CPI_USA', 
                    start_date='1999-12-01', 
                    end_date='2010-12-31')

# # merge the two DataFrames
df_merged = df.join(df_cpi, how='left')

# calculate returns and inflation rate
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.Value.pct_change()

# calculate inflation-adjusted returns

df_merged['real_rtn'] = (df_merged.simple_rtn + 1) / (df_merged.inflation_rate + 1) - 1

# inspect results
df_merged.head()

## Changing frequency

In [ ]:
import pandas as pd 
import fix_yahoo_finance as yf

# download and clean data 
df = yf.download('AAPL', 
                 start='2000-01-01', 
                 end='2010-12-31', 
                 auto_adjust=False,
                 progress=False)

# keeping one important column and renaming
df = df.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close': 'adj_close'}, inplace=True)

# calculate simple returns
df['simple_rtn'] = df.adj_close.pct_change()

# remove redundant data
df.drop('adj_close', axis=1, inplace=True)
df.dropna(axis=0, inplace=True)

# define function for calculating realized volatility over a group
def realized_volatility(x):
    return np.sqrt(np.sum(x**2))

# calculate monthly realized volatility
df_rv = df.groupby(pd.Grouper(freq='M')).apply(realized_volatility)

# rename column
df_rv.rename(columns={'simple_rtn': 'rv'}, inplace=True)

# change frequency
df_rv.rv = df_rv.rv * np.sqrt(12)

# basic plotting
%matplotlib inline
%config InlineBackend.figure_format ='retina'
df.plot();
df_rv.plot();

## Visualising time series data

In [ ]:
import pandas as pd 
import fix_yahoo_finance as yf

# download data as pandas DataFrame
df = yf.download('MSFT', auto_adjust = False)
df = df[['Adj Close']]
df.rename(columns={'Adj Close': 'adj_close'}, inplace=True)

# create simple and log returns
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = (np.log(df.adj_close) - np.log(df.adj_close.shift(1)))

# dropping NA's in the first row
df.dropna(how = 'any', inplace = True)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import matplotlib.pyplot as plt
plt.style.use('seaborn') #set style to `seaborn`

# create placeholder subplots 
fig, ax = plt.subplots(3, 1, figsize=(24, 20))

# add prices
df.adj_close.plot(ax=ax[0])
ax[0].set_ylabel('Stock price ($)', fontsize=14)
ax[0].set_xlabel('')
ax[0].set_title('MSFT time series', fontsize=20)

# add simple returns 
df.simple_rtn.plot(ax=ax[1])
ax[1].set_ylabel('Simple returns (%)', fontsize=14)
ax[1].set_xlabel('')

# add log returns 
df.log_rtn.plot(ax=ax[2])
ax[2].set_ylabel('Log returns (%)', fontsize=14)
ax[2].set_xlabel('Date', fontsize=14)

# display the figure
fig.show()

In [ ]:
import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode

# set up settings (run it once)
#cf.set_config_file(world_readable=True, theme='pearl', offline=True)

init_notebook_mode()

df.iplot(subplots=True, shape=(3,1), shared_xaxes=True, title='MSFT time series')

## Identifying outliers

In [ ]:
%config InlineBackend.figure_format ='retina'
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn') #set style to `seaborn`

# identify outliers using 3 sigma approach ----

#calculate rolling mean and standard deviation
df_ma = df[['simple_rtn']].rolling(window=21).agg(['mean', 'std'])

# drop multi-level index
df_ma.columns = df_ma.columns.droplevel()

# identify outliers
df_outliers = df.join(df_ma)
df_outliers['outlier'] = [1 if (x > mu + 3 * sigma) or (x < mu - 3 * sigma) else 0 for x, mu, sigma in zip(df_outliers.simple_rtn, 
                                                                                                           df_outliers['mean'], 
                                                                                                           df_outliers['std'])] 
# visualize the results ----

# create instance of plot
fig, ax = plt.subplots(figsize=(15, 9))

# define outliers for convenience
outliers = df_outliers.loc[df_outliers['outlier'] == 1, ['simple_rtn']]

# add line plot of returns
ax.plot(df_outliers.index, df_outliers.simple_rtn, color='blue', label='Normal')
# add points for outliers
ax.scatter(outliers.index, outliers.simple_rtn, color='red', label='Anomaly')

# details about the plot 
plt.legend(loc='lower right')
plt.title('Apple stock returns', fontsize = 20)
plt.show();

## Investigating stylized facts of asset returns

In [ ]:
import pandas as pd 
import numpy as np
import fix_yahoo_finance as yf

# download and preprocess data ----
df = yf.download('^GSPC', 
                 start='1985-01-01', 
                 end='2018-12-31',
                 progress=False)

df = df[['Adj Close']].rename(columns={'Adj Close': 'adj_close'})
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))
df = df[['adj_close', 'log_rtn']].dropna(how = 'any')

#### Fact 1 - Non-Gaussian distribution of returns

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import fix_yahoo_finance as yf
import seaborn as sns 
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.tsa.api as smt
plt.style.use('seaborn') #set style to `seaborn`

# plots ----

# create placeholder subplots 
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

# histogram (distribution)
sns.distplot(df.log_rtn, kde=False, norm_hist=True, ax=ax[0])                                    
ax[0].set_title('Distribution of MSFT returns', fontsize=16)

# store range of returns values
r_range = np.linspace(min(df.log_rtn), max(df.log_rtn), num=1000)

# impose a PDF of Normal distribution with sample mean and std  
mu = df.log_rtn.mean()
sigma = df.log_rtn.std()
norm_pdf = scs.norm.pdf(r_range, loc=mu, scale=sigma)                                                         
ax[0].plot(r_range, norm_pdf, 'g', lw=2, label=f'Normal({round(mu, 2)}, {round(sigma, 2)})');

# add legend
ax[0].legend();

# QQ plot
qq = sm.qqplot(df.log_rtn.values, line='s', ax=ax[1])
ax[1].set_title('QQ plot', fontsize = 16)

plt.show()

# Descriptive statistics ----
print('---------- Descriptive Statistics ----------')
print('Range of dates:', min(df.index.date), '-', max(df.index.date))
print('Number of observations:', df.shape[0])
print('Mean: {0:.4f}'.format(df.log_rtn.mean()))
print('Median: {0:.4f}'.format(df.log_rtn.median()))
print('Min: {0:.4f}'.format(df.log_rtn.min()))
print('Max: {0:.4f}'.format(df.log_rtn.max()))
print('Standard Deviation: {0:.4f}'.format(df.log_rtn.std()))
print('Skewness: {0:.4f}'.format(df.log_rtn.skew()))
print('Kurtosis: {0:.4f}'.format(df.log_rtn.kurtosis())) 
print('Jarque-Bera statistic: {stat:.2f} with p-value: {p_val:.2f}'.format(stat = scs.jarque_bera(df.log_rtn.values)[0],
                                                                           p_val = scs.jarque_bera(df.log_rtn.values)[1]))

#### Fact 2 - Volatility Clustering

In [ ]:
df.log_rtn.plot(title='Daily MSFT returns', figsize=(10, 6));

#### Fact 3 - Absence of autocorrelation in returns

In [ ]:
# create autocorrelation plot 
acf = smt.graphics.plot_acf(df.log_rtn, lags=50 , alpha=0.05);

#### Fact 4 - Small and decreasing autocorrelation in squared/absolute returns

In [ ]:
# specify the max amount of lags
lags = 50

# create placeholder subplots 
fig, ax = plt.subplots(2, 1, figsize=(12, 10))

# plot autocorrelation of squared returns 
smt.graphics.plot_acf(df.log_rtn ** 2, lags=lags, alpha=0.05, ax = ax[0])
ax[0].set_ylabel('Squared Returns', fontsize=14)
ax[0].set_title('Autocorrelation Plots', fontsize=20)
ax[0].set_xlabel('')

# plot autocorrelation of absolute returns
smt.graphics.plot_acf(np.abs(df.log_rtn), lags=lags, alpha=0.05, ax = ax[1])
ax[1].set_ylabel('Absolute Returns', fontsize=14)
ax[1].set_title('')
ax[1].set_xlabel('Lag', fontsize=14)
fig.show()

#### Fact 5 - Leverage effect

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import matplotlib.pyplot as plt
plt.style.use('seaborn') #set style to `seaborn`

# calculate volatility measures as moving standard deviations
df['moving_std_252'] = df[['log_rtn']].rolling(window=252).std()
df['moving_std_21'] = df[['log_rtn']].rolling(window=21).std()

# create placeholder subplots 
fig, ax = plt.subplots(3, 1, figsize=(24, 20))

# add prices
df.adj_close.plot(ax=ax[0])
ax[0].set_ylabel('Stock price ($)', fontsize=14)
ax[0].set_xlabel('')
ax[0].set_title('MSFT time series', fontsize=20)

# add simple returns 
df.log_rtn.plot(ax=ax[1])
ax[1].set_ylabel('Log returns (%)', fontsize=14)
ax[1].set_xlabel('')

# add log returns 
df.moving_std_252.plot(ax=ax[2], color='r', label='Moving Volatility 252d')
df.moving_std_21.plot(ax=ax[2], color='g', label='Moving Volatility 21d')
ax[2].set_ylabel('Moving Volatility', fontsize=14)
ax[2].set_xlabel('Date', fontsize=14)
ax[2].legend()

# display the figure
fig.show()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import pandas as pd 
import numpy as np
import fix_yahoo_finance as yf
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn') #set style to `seaborn`

# download and preprocess data ----
df = yf.download(['^GSPC', '^VIX'], 
                 start='1985-01-01', 
                 end='2018-12-31',
                 progress=False)
df = df[['Adj Close']]
df.columns = df.columns.droplevel(0)
df = df.rename(columns={'^GSPC': 'sp500', '^VIX': 'vix'})
df['log_rtn'] = np.log(df.sp500/df.sp500.shift(1))
df['vol_rtn'] = np.log(df.vix/df.vix.shift(1))
df.dropna(how='any', axis=0, inplace=True)

sns.regplot(x='log_rtn', y='vol_rtn', data=df, line_kws={'color': 'red'}) \
   .set_title(f'SP500 returns vs. VIX returns ($\\rho$ = {df.log_rtn.corr(df.vol_rtn).round(2)})', fontsize=16)

plt.xlabel('SP500 log returns')
plt.ylabel('VIX log returns');